# Introduction #

# Prepare Data for Scoring #

Equality check.

# Create Mutual Information Scores #

Equality check.

# Apply Permutation Test #

Thought question.

# Choose Features for Development #

Thought question.

# Conclusion #